<a href="https://colab.research.google.com/github/talhaanwarch/Sarcasm-nlp/blob/master/reddit_multi_input_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [0]:
!pip install tensorflow==2.1.0
!pip install sentencepiece
!pip install tensorflow-hub
!pip install tensorflow-addons
#!wget --quiet https://raw.githubusercontent.com/google-research/ALBERT/master/tokenization.py

In [0]:
cd /content/drive/My\ Drive/dataset/sarcasm

In [0]:
ls

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score


In [0]:
reddit_train=pd.read_json('sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)

In [0]:
reddit_train.head()

In [0]:
#reddit_train['response']=reddit_train['response'].str.replace('@USER', "") 
reddit_train['response']=reddit_train['response'].str.replace('\d+', '')
reddit_train['response']=reddit_train['response'].str.lower()
reddit_train['response']=reddit_train['response'].str.replace('[^\w\s]','')


In [0]:
reddit_train.head()

In [0]:
reddit_train['context']=reddit_train['context'].apply(lambda x: ','.join(map(str, x)))
#reddit_train['context']=reddit_train['context'].str.replace('@USER', "") 
reddit_train['context']=reddit_train['context'].str.lower()
reddit_train['context']=reddit_train['context'].str.replace('[^\w\s]','')
reddit_train['context']=reddit_train['context'].str.replace('\d+', '')


In [0]:
reddit_train.head()

In [0]:
len(reddit_train)

In [0]:
from collections import Counter
Counter(reddit_train['label'])

In [0]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_label=le.fit_transform(reddit_train['label'])

In [0]:
reddit_test=pd.read_json('sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)
reddit_test.head()

In [0]:
#reddit_test['response']=reddit_test['response'].str.replace('@USER', "") 
reddit_test['response']=reddit_test['response'].str.replace('\d+', '')
reddit_test['response']=reddit_test['response'].str.lower()
reddit_test['response']=reddit_test['response'].str.replace('[^\w\s]','')
reddit_test['context']=reddit_test['context'].apply(lambda x: ','.join(map(str, x)))
#reddit_test['context']=reddit_test['context'].str.replace('@USER', "") 
reddit_test['context']=reddit_test['context'].str.lower()
reddit_test['context']=reddit_test['context'].str.replace('[^\w\s]','')
reddit_test['context']=reddit_test['context'].str.replace('\d+', '')


In [0]:
reddit_test.head()

#BERT

In [0]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
import tokenization
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
max_len=200

In [0]:
train_response = bert_encode(reddit_train.response, tokenizer, max_len=max_len)
train_context = bert_encode(reddit_train.context, tokenizer, max_len=max_len)
train_labels=reddit_train.label.values
#test_input = bert_encode(test_bert.values, tokenizer, max_len=160)

In [0]:
test_response = bert_encode(reddit_test.response, tokenizer, max_len=max_len)
test_context = bert_encode(reddit_test.context, tokenizer, max_len=max_len)

In [0]:
test_gen=[test_context[0],test_context[1],test_context[2],test_response[0],test_response[1],test_response[2]]

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,concatenate,Dense,GlobalAveragePooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa

In [0]:
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)

# ...

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

In [0]:
contex_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_word_ids")
contex_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="contex_input_mask")
contex_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="contex_segment_ids")
_, context_sequence_output = bert_layer([contex_input_word_ids, contex_input_mask, contex_segment_ids])
context_clf_output = context_sequence_output[:, 0, :]


reponse_input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_word_ids")
reponse_input_mask = Input(shape=(max_len,), dtype=tf.int32, name="reponse_input_mask")
reponse_segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="reponse_segment_ids")
_, reponset_sequence_output = bert_layer([reponse_input_word_ids, reponse_input_mask, reponse_segment_ids])
reponset_clf_output = reponset_sequence_output[:, 0, :]



conc=concatenate([context_clf_output,reponset_clf_output])
conc=Dropout(0.3)(conc)
out = Dense(1, activation='sigmoid')(conc)
    
model = Model(inputs=[contex_input_word_ids, contex_input_mask, contex_segment_ids,reponse_input_word_ids,reponse_input_mask,reponse_segment_ids], outputs=out)
model.compile(Adam(1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    


In [0]:
train_gen=[train_context[0],train_context[1],train_context[2],train_response[0],train_response[1],train_response[2]]

In [0]:
train_history = model.fit(
    train_gen, train_label,
    epochs=10,
    batch_size=4)

In [0]:
#len(train_gen),len(train_label)

In [0]:
#train_context[0].shape

In [0]:
# import tensorflow_addons as tfa
# from sklearn.model_selection import StratifiedKFold


In [0]:
# def unpack(a, b, c, d,e,f,g): 
#     #print(a[g].shape, b[g].shape, c[g].shape, d[g].shape,e[g].shape,f[g].shape) 
#     return [a[g], b[g], c[g], d[g],e[g],f[g]]

In [0]:
#unpack(*train_gen,train)

In [0]:
# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=5, shuffle=False)
# scores = []
# test=[]
# s=0
# for train, val in kfold.split(list(zip(*train_gen)), train_label):
#   model.fit(unpack(*train_gen,train),train_label[train],batch_size=4,epochs=5,verbose=2,
#             validation_data=(unpack(*train_gen,val), train_label[val]))
   

#   y_pred = model.predict(unpack(*train_gen,val), batch_size=4, verbose=1)

#   test.append(model.predict(test_gen, batch_size=4, verbose=1).ravel())

#   y_pred = (y_pred > 0.5)
#   f1=f1_score(train_label[val], y_pred, average='macro')
#   s+=1
#   print(s,' f1 score  is ',f1)
#   scores.append(f1)
#   tf.keras.backend.clear_session()